In [20]:
from PIL import Image
import requests
from transformers import ChineseCLIPProcessor, ChineseCLIPModel

model = ChineseCLIPModel.from_pretrained("/Users/zhangyf/llm/chinese-clip-vit-base-patch16")
processor = ChineseCLIPProcessor.from_pretrained("/Users/zhangyf/llm/chinese-clip-vit-base-patch16")

url = "https://clip-cn-beijing.oss-cn-beijing.aliyuncs.com/pokemon.jpeg"
# image = Image.open(requests.get(url, stream=True).raw)
image = Image.open("1.jpg")
# Squirtle, Bulbasaur, Charmander, Pikachu in English
texts = ["两只老虎在雪地里", "两只狗在雪地里", "小火龙", "一只皮丘"]

# 计算图像的特征
inputs = processor(images=image, return_tensors="pt")
image_features = model.get_image_features(**inputs) # 使用clip模型中的图像编码器抽取特征
image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)  # normalize

# 计算文本特征
inputs = processor(text=texts, padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs) # 使用clip模型中的文本编码器抽取特征
text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)  # normalize

# 计算相似度分数
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # probs: [[1.2686e-03, 5.4499e-02, 6.7968e-04, 9.4355e-01]]


In [21]:
import torch
_,index = torch.max(probs, dim=1)
print(texts[index.item()])

两只狗在雪地里
